In [4]:
!kaggle competitions download - c kdtai-2


usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...
kaggle: error: unrecognized arguments: c kdtai-2


In [5]:
!unzip kdtai-2.zip - d dataset


unzip:  cannot find or open kdtai-2.zip, kdtai-2.zip.zip or kdtai-2.zip.ZIP.


In [6]:
%pip install nltk
%pip install gensim
%pip install soynlp


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os


In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/lee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
device = torch.device("mps")
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: mps


In [10]:
def preprocessing(df):
    pass

# Preprocessing 과정을 분리해야 합니다.


In [23]:
from enum import Enum
from nltk.tokenize import word_tokenize
from soynlp.hangle import decompose, character_is_korean
import re
from tqdm import tqdm


class Dataset_type(Enum):
    TRAIN = 0
    TEST = 1


class Korean_dataset(Dataset):
    def __init__(self, file_path, dataset_type: Dataset_type, embedding_model, is_split_jamo=False):
        super().__init__()
        self.file_path = file_path
        # self.transform = transform
        self.dataset_type = dataset_type
        self.data_df = pd.read_csv(self.file_path)
        self.embedding_model = embedding_model
        self.is_split_jamo = is_split_jamo
        self.embedded_src = self._embedding(self.data_df)

    def __len__(self):
        return len(self.embedded_src)

    def __getitem__(self, idx):
        src = self.embedded_src[idx]
        if self.dataset_type == Dataset_type.TRAIN:
            trg = torch.tensor(self.data_df.loc[idx, "label"]).to(device)
            return src, trg
        else:
            return src

    def _embedding(self, df):
        src_list = []
        rows_to_drop = []
        # preprocessed_df = df.copy()
        for idx, text in enumerate(df.loc[:, "text"]):
            text = self._remove_special_characters(text)
            src = word_tokenize(text)
            if not src:
                rows_to_drop.append(idx)
                continue
            src = [self._split_jamo(word)
                   for word in src] if self.is_split_jamo else src
            src = [self.embedding_model.wv[word] for word in src]
            src = torch.FloatTensor(src).to(device)
            src_list.append(src)
        df.drop(rows_to_drop, inplace=True)
        df = df.reset_index(drop=True, inplace=True)
        return src_list

    def _remove_special_characters(self, text):
        return re.sub(r'[^\w\s]', '', text)

    def _split_jamo(self, word):

        def transform(char):
            if char == ' ':
                return char
            cjj = decompose(char)
            if len(cjj) == 1:
                return cjj
            cjj_ = ''.join(c if c != ' ' else '-' for c in cjj)
            return cjj_

        sent_ = []
        for char in word:
            if character_is_korean(char):
                sent_.append(transform(char))
            else:
                sent_.append(char)
        doublespace_pattern = re.compile('\s+')
        sent_ = doublespace_pattern.sub(' ', ''.join(sent_))
        return sent_

In [14]:
import gensim
current_path = os.getcwd()
model_file_path = os.path.join(current_path, "embedding_model", "wiki.ko.bin")
embedding_model = gensim.models.fasttext.load_facebook_model(model_file_path)

In [24]:
from torch.utils.data import random_split

current_path = os.getcwd()
train_file_path = os.path.join(current_path, "dataset", "train.csv")
test_file_path = os.path.join(current_path, "dataset", "test.csv")

train_set = Korean_dataset(file_path=train_file_path,
                           dataset_type=Dataset_type.TRAIN,
                           is_split_jamo=False, embedding_model=embedding_model)
test_set = Korean_dataset(file_path=test_file_path,
                          dataset_type=Dataset_type.TEST,
                          is_split_jamo=False, embedding_model=embedding_model)


# train_valid_ratio = 0.9
# train_set_count = int(len(train_set) * train_valid_ratio)
# val_set_count = len(train_set) - train_set_count
# train_set, val_set = random_split(train_set, [train_set_count, val_set_count])
# print(len(train_set))
# print(len(val_set))
# print(len(test_set))

In [ ]:
print(len(train_set))


65835


In [ ]:
train_set[65833]


(tensor([[-0.0718,  0.0064, -0.1133,  ..., -0.0575, -0.0191, -0.1486],
         [-0.0041,  0.0910, -0.2717,  ..., -0.0592, -0.1021, -0.1022],
         [ 0.0286,  0.0155, -0.0673,  ...,  0.0514, -0.0640,  0.1243],
         [-0.0167,  0.0737, -0.2717,  ...,  0.0057,  0.0109,  0.1132]]),
 2)

In [27]:
from torch.nn.utils.rnn import pad_sequence


class LSTM_Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1):
        super(LSTM_Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim,
                            num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0),
                         self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0),
                         self.hidden_dim).to(x.device)
        out, (hn, cn) = self.lstm(x, (h0, c0))
        # print(hn.shape)
        # print(hn.squeeze(0).shape)
        hn_last = hn[-1]  # 마지막 레이어의 hidden state 선택
        x = self.fc(hn_last)
        return x


input_dim = 300
hidden_dim = 128
output_dim = 7
num_layers = 1
batch_size = 64
num_of_epoch = 20
learning_rate = 0.0005

model = LSTM_Net(input_dim=input_dim, hidden_dim=hidden_dim,
                 output_dim=output_dim, num_layers=num_layers).to(device)

# 손실 함수와 최적화 알고리즘을 정의합니다.
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


def train(dataset, valid_rate, batch_size, optimizer, num_of_epoch, valid_random_seed=None):

    def collate_fn(batch):
        inputs = [item[0] for item in batch]
        targets = [item[1] for item in batch]

        # 입력 데이터의 길이를 기준으로 내림차순으로 정렬
        sorted_idx = sorted(range(len(inputs)),
                            key=lambda i: len(inputs[i]), reverse=True)
        inputs = [inputs[i] for i in sorted_idx]
        targets = [targets[i] for i in sorted_idx]

        # 패딩된 새로운 텐서 생성
        inputs = pad_sequence(inputs, batch_first=True, padding_value=0.0)
        # print(inputs.shape)

        return inputs, torch.tensor(targets)

    dataset = dataset
    train_set_count = int(len(dataset) * valid_rate)
    val_set_count = len(dataset) - train_set_count

    if valid_random_seed is not None:
        torch.manual_seed(valid_random_seed)

    train_set, val_set = random_split(
        dataset, [train_set_count, val_set_count])
    train_loader = DataLoader(train_set, batch_size=batch_size,
                              shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_set, batch_size=batch_size,
                            shuffle=True, collate_fn=collate_fn)
    optimizer = optimizer
    criterion = nn.CrossEntropyLoss().to(device)
    train_losses = []
    train_accs = []
    val_losses = []
    val_accs = []

    for epoch in tqdm(range(num_of_epoch)):
        model.train()
        correct_train = 0
        for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            _, pred = torch.max(output.data, 1)
            correct_train += pred.eq(target.view_as(pred)).sum().item()

            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch + 1, batch_idx *
                    len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

        train_accuracy = 100. * (correct_train / len(train_set))
        train_losses.append(loss.item())
        train_accs.append(train_accuracy)
        print(f"Epoch: {epoch + 1} - train accuracy: {train_accuracy}")

        model.eval()
        correct_val = 0
        with torch.no_grad():
            for data, target in tqdm(val_loader):
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)
                _, pred = torch.max(output.data, 1)
                correct_val += pred.eq(target.view_as(pred)).sum().item()

            val_accuracy = 100. * (correct_val / len(val_set))
            val_losses.append(loss.item())
            val_accs.append(val_accuracy)

            print('Epoch {} finished: train loss = {}, val loss = {}'.format(epoch + 1,
                                                                             train_losses[-1], val_losses[-1]))
            print(f"Epoch: {epoch + 1} - Validation accuracy: {val_accuracy}")


In [ ]:
train(dataset=train_set, valid_rate=0.9, batch_size=batch_size,
      optimizer=optimizer, num_of_epoch=num_of_epoch, valid_random_seed=42)

  0%|          | 0/20 [00:00<?, ?it/s]

Train Epoch: 1 [0/59251 (0%)]	Loss: 1.962359


Train Epoch: 1 [6400/59251 (11%)]	Loss: 1.751284


Train Epoch: 1 [12800/59251 (22%)]	Loss: 1.895719


Train Epoch: 1 [19200/59251 (32%)]	Loss: 1.586071


...

Train Epoch: 20 [57600/59251 (97%)]	Loss: 0.477946


926it [00:37, 24.45it/s]


Epoch: 20 - train accuracy: 81.41128419773506


100%|██████████| 20/20 [13:29<00:00, 40.45s/it]

Epoch 20 finished: train loss = 0.6100606918334961, val loss = 1.0738309621810913
Epoch: 20 - Validation accuracy: 70.18529769137302


In [26]:
train(dataset=train_set, valid_rate=0.9, batch_size=batch_size,
      optimizer=optimizer, num_of_epoch=num_of_epoch, valid_random_seed=42)

0it [00:01, ?it/s]/20 [00:00<?, ?it/s]
  0%|          | 0/20 [00:01<?, ?it/s]


RuntimeError: Placeholder storage has not been allocated on MPS device!

In [17]:
!conda env config vars set PYTORCH_ENABLE_MPS_FALLBACK = 1


To make your changes take effect please reactivate your environment


In [22]:
!echo $PYTORCH_ENABLE_MPS_FALLBACK


1
